# PAM Read-Modify-Write

This notebook is an introduction to the basic read - modify - write use case of PAM:

- Read: Load activity plans from existing data (either tabular or MATSim)
- Modify: Use the PAM api to modify the activity plans
- Write: Write activity plans back to disk in the chosen format

For this example, we use policies to make our modifications. But you might also try the following:

- spatial sampling
- location modelling
- rescheduling
- adding noise
- simulating aging or the passing of time
- and so on...

In [1]:
import os
from collections import defaultdict

import geopandas as gp
import pandas as pd
from matplotlib import pyplot as plt
from pam import policy, read
from pam.policy import apply_policies

%matplotlib inline

ModuleNotFoundError: No module named 'pam'

## Load Data

Here we load simple travel diary data of London commuters. This is a very simple 0.1% sample of data about work and education commutes from the 2011 census. 
Because we're sharing this data, we've aggregated locations to borough level and randomized personal attributes; so, don't get too excited about the results.

The data is available in the [`data/example_data`](https://github.com/arup-group/pam/tree/main/examples/data/example_data) sub-directory.
All data paths in this example are relative to the [notebook directory](https://github.com/arup-group/pam/tree/main/examples) in the PAM repository 

In [2]:
trips = pd.read_csv(
    os.path.join("data", "example_data", "example_travel_diaries.csv"), index_col="uid"
)
attributes = pd.read_csv(
    os.path.join("data", "example_data", "example_attributes.csv"), index_col="pid"
)

In [3]:
trips.head(10)

## Read

First we load example travel diary data to Activity Plans. This data represents 2011 baseline London population of commuters.

In [4]:
population = read.load_travel_diary(trips, attributes, trip_freq_as_person_freq=True)

Let's check out an example Activity Plan and Attributes:

In [5]:
household = population.households["census_12"]
person = household.people["census_12"]
person.print()

Before we do any activity modification - we create a simple function to extract some example statistics. We include this as a simple demo, but would love to add more.

Note that activity plans allow us to consider detailed **joint** segmentations, such as socio-economic, spatial, temporal, modal, activity sequence and so on.

In [6]:
def print_simple_stats(population):
    """Print some simple population statistics."""
    time_at_home = 0
    travel_time = 0
    low_income_central_trips = 0
    high_income_central_trips = 0

    for hh in population.households.values():
        for person in hh.people.values():
            freq = person.freq

            for p in person.plan:
                if p.act == "travel":
                    duration = p.duration.seconds * freq / 3600
                    travel_time += duration

                    if p.end_location.area == "Westminster,City of London":
                        if person.attributes["inc"] == "low":
                            low_income_central_trips += freq

                        elif person.attributes["inc"] == "high":
                            high_income_central_trips += freq

                else:  # activity
                    if p.act == "home":
                        duration = p.duration.seconds * freq / 3600
                        time_at_home += duration

    print(f"Population total time at home: {time_at_home/1000000:.2f} million hours")
    print(f"Population total travel time: {travel_time/1000000:.2f} million hours")
    print(f"Low income trips to Central London: {low_income_central_trips} trips")
    print(f"High income trips to Central London: {high_income_central_trips} trips")

In [7]:
print_simple_stats(population)

In [8]:
def plot_simple_stats(population):
    """Plot some simple population statistics."""
    geoms = gp.read_file(os.path.join("data", "example_data", "geometry.geojson"))

    departures = defaultdict(int)
    arrivals = defaultdict(int)

    for _hid, hh in population.households.items():
        for _pid, person in hh.people.items():
            freq = person.freq

            for p in person.plan:
                if p.act == "travel":
                    departures[p.start_location.area] += freq
                    arrivals[p.end_location.area] += freq
    geoms["departures"] = geoms.NAME.map(departures)
    geoms["arrivals"] = geoms.NAME.map(arrivals)

    fig, ax = plt.subplots(1, 2, figsize=(16, 6))
    for i, name in enumerate(["departures", "arrivals"]):
        ax[i].title.set_text(name)
        geoms.plot(name, ax=ax[i])
        ax[i].axis("off")

In [9]:
plot_simple_stats(population)

## Modify

Our 2011 baseline London population of commuters seems sensible, they spend about 50 million hours at home and 1.6 million hours travelling.

But what if we want to try and build some more up to date scenarios?

We consider two scenarios from a combination of policies:

Scenario A - Do Minimum:
1. A household will be quarantined with p=0.025 (for example due to a possitive virus test within the household)
2. A person will be staying at home (self isolating) with p=0.1 (for example due to being a vulnerable person)

Scenario B - Lockdown:
1. As above plus education and work activities will be removed and plans adjusted with p=0.9 (for example because schools and work places are closed)

In [10]:
policy1 = policy.HouseholdQuarantined(probability=0.025)
policy2 = policy.PersonStayAtHome(probability=0.1)
policy3 = policy.RemoveHouseholdActivities(["education", "work"], probability=0.9)

do_minimum = apply_policies(population, [policy1, policy2])
lockdown = apply_policies(population, [policy1, policy2, policy3])

In [11]:
print_simple_stats(do_minimum)
plot_simple_stats(do_minimum)

In [12]:
print_simple_stats(lockdown)
plot_simple_stats(lockdown)

## Write

Assuming we are happy with our modified activity sequences we can write them to disk in our desired format. For this example we haven't prepared the population for MATSim so we write to disk as travel plans/diaries:

In [13]:
do_minimum.to_csv(os.path.join("tmp", "do_min"))
lockdown.to_csv(os.path.join("tmp", "lockdown"))